# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][HW04] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [35]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from math import sqrt
from catboost import CatBoostClassifier


ModuleNotFoundError: No module named 'catboost'

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
class DecisionTreeNode:
    def __init__(self, dim, left, right):
        self.dim = dim
        self.left = left
        self.right = right


class DecisionTreeLeaf:
    def __init__(self, y):
        self.y = y
        labels, count = np.unique(y, return_counts=True)
        self.label = labels[np.argmax(count)]


class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features=None):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        if max_features is None:
            self.max_features = int(sqrt(len(X[0])))
        if criterion == "gini":
            self.criterion = gini
        if criterion == "entropy":
            self.criterion = entropy
        self.root = self.makeNode(np.array(list(zip(y, X))), 0)

    def gain(self, left_y, right_y, criterion):
        if len(left_y) < self.min_samples_leaf or len(right_y) < self.min_samples_leaf:
            return -1
        else:
            return gain(left_y, right_y, criterion)

    def makeNode(self, X_y, depth):
        if depth == self.max_depth or len(X_y) < 2 * self.min_samples_leaf:
            return DecisionTreeLeaf(X_y[:, 0])
        all_features = np.array(list(range(len(X_y[0][1]))))
        np.random.shuffle(all_features)
        features = all_features[:self.max_features]
        gains = [self.gain(np.array([z[0] for z in filter(lambda x: x[1][d] == 0, X_y)]),
                           np.array([z[0] for z in filter(lambda x: x[1][d] == 1, X_y)]),
                           self.criterion) for d in features]
        dim = features[np.argmax(gains)]
        max_gain = max(gains)
        if max_gain <= 0:
            return DecisionTreeLeaf(X_y[:, 0])
        else:
            left = np.array(list(filter(lambda x: x[1][dim] == 0, X_y)))
            right = np.array(list(filter(lambda x: x[1][dim] == 1, X_y)))
            return DecisionTreeNode(dim, self.makeNode(left, depth + 1),
                                    self.makeNode(right, depth + 1))

    def find_x(self, tree, x):
        if isinstance(tree, DecisionTreeLeaf):
            return tree.label
        else:
            if x[tree.dim] == 0:
                return self.find_x(tree.left, x)
            else:
                return self.find_x(tree.right, x)

    def predict(self, X):
        return [self.find_x(self.root, x) for x in X]


### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [17]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10,
                 ratio=1.0):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        if max_features == "auto":
            self.max_features = int(sqrt(len(X[0])))
        self.n_estimators = n_estimators
        self.ratio = ratio
        self.X = None
        self.y = None
        self.trees = None
        self.oob = None
        self.m = None

    def importance(self):
        importance = np.array([0.0 for j in range(len(X[0]))])
        for n in range(self.n_estimators):
            X_oob = np.array([self.X[i] for i in self.oob[n]])
            y_oob = np.array([self.y[i] for i in self.oob[n]])
            err_obb = len(list(filter(lambda x: x[0] == x[1], zip(self.trees[n].predict(X_oob), y_oob)))) / len(y_oob)
            for j in range(len(X[0])):
                col_j = copy.copy(X_oob[:, j])
                np.random.shuffle(col_j)
                X_oob_j = copy.copy(X_oob)
                X_oob_j[:, j] = col_j
                err_j = len(list(filter(lambda x: x[0] == x[1], zip(self.trees[n].predict(X_oob_j), y_oob)))) / len(
                    y_oob)
                importance[j] += err_obb - err_j
        return importance / self.n_estimators

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.m = int(self.ratio * len(X))

        def baggingTree():
            randind = np.random.randint(len(X), size=self.m)
            tree = DecisionTree([X[i] for i in randind], [y[i] for i in randind], criterion=self.criterion,
                                min_samples_leaf=self.min_samples_leaf,
                                max_depth=self.max_depth, max_features=self.max_features)
            out_of_bag = np.array(list(filter(lambda x: x not in randind, range(len(X)))))
            return tree, out_of_bag

        trees_oob = np.array([baggingTree() for n in range(self.n_estimators)])
        self.trees = trees_oob[:, 0]
        self.oob = trees_oob[:, 1]

    def predict(self, X):
        preds = np.transpose(np.array([tree.predict(X) for tree in self.trees]))

        def best(predictions):
            labels, count = np.unique(predictions, return_counts=True)
            return labels[np.argmax(count)]

        return [best(y) for y in preds]



### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [15]:
def feature_importance(rfc):
    return rfc.importance()


def most_important_features(importance, names, k=20):
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [18]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), int(i % 6 == 3), 
          int(i % 6 == 0), int(i % 3 == 2), np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [ 0.00083672 -0.00086362  0.15637837  0.15830291  0.3348981   0.00282328]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [19]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [20]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [21]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7414880201765448
Most important features:
1. ovsyanochan
2. 4ch
3. rhymes
4. styd.pozor
5. mudakoff
6. dayvinchik
7. rapnewrap
8. reflexia_our_feelings
9. bot_maxim
10. pravdashowtop
11. iwantyou
12. pixel_stickers
13. pozor
14. tumblr_vacuum
15. leprum
16. i_des
17. ohhluul
18. exclusive_muzic
19. rem_shkola
20. ne.poverish


#### Пол

In [22]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8511979823455234
Most important features:
1. 40kg
2. modnailru
3. 9o_6o_9o
4. girlmeme
5. recipes40kg
6. mudakoff
7. 4ch
8. be.women
9. femalemem
10. zerofat
11. bot_maxim
12. cook_good
13. i_d_t
14. igm
15. be.beauty
16. reflexia_our_feelings
17. rapnewrap
18. woman.blog
19. thesmolny
20. sh.cook


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [36]:
X, y = synthetic_dataset(1000)
cbc = CatBoostClassifier(iterations=100,
                         loss_function='MultiClass',
                         task_type="GPU"
                         )
cbc.fit(X, y)
print("Accuracy:", np.mean(cbc.predict(X).reshape((len(y))) == y))
print("Importance:", cbc.get_feature_importance())

NameError: name 'CatBoostClassifier' is not defined

### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [37]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [38]:
cbc = CatBoostClassifier(iterations=100,
                         loss_function='MultiClass',
                         task_type="GPU"
                         )
cbc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(cbc.predict(X_test).reshape((len(y_age_test))) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(cbc.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

NameError: name 'CatBoostClassifier' is not defined

#### Пол

In [39]:
cbc = CatBoostClassifier(iterations=100,
                         loss_function='MultiClass',
                         task_type="GPU"
                         )
cbc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(cbc.predict(X_test).reshape((len(y_sex_test))) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(cbc.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

NameError: name 'CatBoostClassifier' is not defined